In [1]:
from stage import getStage
import pandas as pd
import datetime
from datetime import date, datetime
import yahoo_fin.stock_info as yf

In [2]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
goodSectorDf = sectorDfList[0].drop(['close','ticker','MA30','MA30Slope','RS'],axis=1)
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf.index = sectorDfList[0].index
for date in sectorDfList[0].index:
    listOfSector = []
    for df in sectorDfList:
        try:
            if df.at[date,'MA30Slope'] > 1.268490629719207 and df.loc[date,'RS'] > 1.0013386634337298:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    listOfSector.append(df.at[date,'ticker'])
        except: 
            pass
    goodSectorDf.at[date,'Sectors'] = set(listOfSector)

In [8]:
# Open a file
result = open("Testdata/Realtime/result"+date.today().strftime('%Y-%m-%d')+".txt", "w")
for symbol in yf.tickers_nasdaq():
    df = getStage(symbol,[0.03361077383204303, 1.7466372328111242, -0.08498841770875612, 0.926192276854965, 1.1764676012556523, 1.1405501342698643, 0.8517616225271734, 0.8691969716399011, 1.268490629719207, 1.0013386634337298, 0.07551020458494859, 0.9489349345040117, 0.996709764664632, 0.9485725801058837, 0.9857019510456804],goodSectorDf)
    shares = 0
    sharesSPY = 0
    if df.empty:
        continue
    if df.iloc[-1].Stage=="Buy":
        print(symbol+" buy")
        #result.write(symbol+" buy\n")
    elif df.iloc[-1].Stage=="Stage 2":
        print(symbol+" hold")
        #result.write(symbol+" hold\n")
    elif df.iloc[-1].Stage=="Sell":
        print(symbol+" sell")
        #result.write(symbol+" sell\n")
